# Lab 1: Logical Agents

In this lab we will learn how to use propositional logic and knowledge bases to create a simple agent to traverse the Wumpus World.

The lab includes:
- Implementing a way to create propositional logic sentences.
- Implementing a function that, given the values of the variables, evaluates the truth-value of a logic sentence.
- Implementing a function that takes a query sentence and checks its truth-value in a given knowledge base by testing all possible models in that knowledge base.
- Implementing an agent with a knowledge base of the Wumpus World, can query that knowledge base for which tiles are safe to visit.

The content of this lab relates strongly to the material of Chapter 6 (Logical Agents) in the course book (Artificial Intelligence: A Modern Approach).
The environment we will tackle in this lab is called the Wumpus World and is covered in the course book as well (Chapter 6.2)

## Implementation of the Wumpus World

Below you will find a code implementation of the Wumpus World along with a few helpful definitions and functions.
The Wumpus World is fully implemented since this lab is about building a propositional logic knowledge base system to build an AI around.
However, since the AI will interact with the Wumpus World, being familiar with the code and functions will help you use them later.

In [37]:
#Imports
import random
import itertools

# Directions
RIGHT = 0
DOWN  = 1
LEFT  = 2
UP    = 3

# How much the agent will move if it takes a step forward for each direction
direction_step = [(1,0), (0,-1), (-1,0), (0,1)]

# Objects
AGENT   = "a"
WUMPUS  = "w"
PIT     = "p"
GOLD    = "g"

# Actions
NONE = ""
FORWARD = "forward"
TURN_LEFT = "turn_left"
TURN_RIGHT = "turn_right"
GRAB = "grab"
SHOOT = "shoot"
CLIMB = "climb"

class WumpusWorld:

    def __init__(self):
        self.game_over = False
        self.agent_dir = RIGHT
        self.agent_has_arrow = True
        self.agent_has_gold = False
        self.wumpus_screams = False
        self.non_start_squares = list(itertools.product([0,1,2,3],[0,1,2,3]))
        self.non_start_squares.pop(0)
        self.cave = [[[],[],[],[]],[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
        self.cave[0][0].append(AGENT)
        self.agent_pos = (0,0)
        self.random_place(WUMPUS)
        self.random_place(GOLD)
        for square in self.non_start_squares:
            if random.random() < 0.2:
                self.cave[square[0]][square[1]].append(PIT)
        self.performance = 0
        self.last_action = NONE

    def random_place(self,object):
        '''
        Randomly places an object into a tile other than the start.
        Args:
            object (any): The object (typically WUMPUS or GOLD) to place
        '''
        pos = random.choice(self.non_start_squares)
        self.cave[pos[0]][pos[1]].append(object)

    def show(self):
        '''
        Prints the Wumpus World cave and current agent performance.
        '''
        for y in range(3,-1,-1):
            for x in range(0,4):
                [print(o,end="") for o in self.cave[x][y]]
                print("\t",end="")
            print()
        print(f"Performance: {self.performance}")

    def actuate(self, action):
        '''
        If the game is still running, the input action is performed and the game state is moved forward.
        Args:
            action (str): The action the agent should perform
        '''
        # If the game is over, nothing can be done
        if self.game_over:
            return

        # store the action that was performed, decrease the performance, and remove scream from memory
        self.last_action = action
        self.performance = self.performance-1
        self.wumpus_screams = False

        # If the NONE action is performed, do nothing
        if action == NONE:
            return

        # FORWARD action: Move the agent one tile in its current direction.
        # End the game and lower performance if it steps into a PIT or WUMPUS
        if action == FORWARD:
            x, y = self.agent_pos
            self.cave[x][y].remove(AGENT)
            xp, yp = direction_step[self.agent_dir]
            x = min(max(x+xp,0),3)
            y = min(max(y+yp,0),3)
            self.agent_pos = (x,y)
            if WUMPUS in self.cave[x][y] or PIT in self.cave[x][y]:
                self.game_over = True
                self.performance = self.performance-1000
            else:
                self.cave[x][y].append(AGENT)
            return

        # TURN_LEFT action: Change the agents direction 90 degrees counter-clockwise
        if action == TURN_LEFT:
            self.agent_dir = (self.agent_dir-1)%4
            return
        # TURN_RIGHT action: Change the agents direction 90 degrees clockwise
        if action == TURN_RIGHT:
            self.agent_dir = (self.agent_dir+1)%4
            return
        # GRAB action: If the agent is in the same tile as the gold, remove it from the cave and add it to the agent
        if action == GRAB:
            x, y = self.agent_pos
            if GOLD in self.cave[x][y]:
                self.agent_has_gold = True
                self.cave[x][y].remove(GOLD)
            return
        # SHOOT action: If the agent has the arrow,
        #    remove it, decrease performance, and,
        #    if there is a WUMPUS infront of the agent, remove it and add scream perception
        if action == SHOOT:
            if not self.agent_has_arrow:
                return
            self.agent_has_arrow = False
            self.performance = self.performance-10
            x, y = self.agent_pos
            if self.agent_dir == RIGHT:
                hit_squares = [(min(x+1,3),y),(min(x+2,3),y),(min(x+3,3),y)]
            elif self.agent_dir == DOWN:
                hit_squares = [(x,max(y-1,0)),(x,max(y-2,0)),(x,max(y-3,0))]
            elif self.agent_dir == LEFT:
                hit_squares = [(max(x-1,0),y),(max(x-2,0),y),(max(x-3,0),y)]
            elif self.agent_dir == UP:
                hit_squares = [(x,min(y+1,3)),(x,min(y+2,3)),(x,min(y+3,3))]
            for x, y in hit_squares:
                if WUMPUS in self.cave[x][y]:
                    self.cave[x][y].remove(WUMPUS)
                    self.wumpus_screams = True
            return

        # CLIMB action: If the agent is at the starting tile, end the game and add performance if the agent has the gold
        if action == CLIMB:
            if self.agent_pos == (0,0):
                if self.agent_has_gold:
                    self.performance = self.performance+1000
                self.cave[0][0].remove(AGENT)
                self.agent_pos = None
                self.agent_dir = None
                self.game_over = True
            return

    def perceive(self):
        '''
        Returns for each percept whether that percept was sensed after the last action.
        '''
        x,y = self.agent_pos
        stench = False
        breeze = False
        for sx,sy in adjacent_tiles(x,y):
            if WUMPUS in self.cave[sx][sy]:
                stench = True
            if PIT in self.cave[sx][sy]:
                breeze = True
        glitter = GOLD in self.cave[x][y]
        bump = False
        if self.last_action == FORWARD:
            if self.agent_dir == RIGHT and x == 3:
                bump = True
            elif self.agent_dir == DOWN and y == 0:
                bump = True
            elif self.agent_dir == LEFT and x == 0:
                bump = True
            elif self.agent_dir == UP and y == 3:
                bump = True
        scream = self.wumpus_screams
        return (stench, breeze, glitter, bump, scream)

def adjacent_tiles(x, y):
    '''
    Helper function that, given a position, returns all tiles that are adjacent to it.
    Args:
        x (int): The x coordinate of the position
        y (int): The y coordinate of the position
    '''
    tiles = []
    for sx,sy in [(x-1,y),(x+1,y),(x,y-1),(x,y+1)]:
        if sx<4 and sx>-1 and sy<4 and sy>-1:
            tiles.append((sx,sy))
    return tiles

## Making Propositional Logic
Before we get started with making an AI for the Wumpus World agent, we need to implement propositional logic which will be the foundation of our AI.
Initially it might seem strange to implement propositional logic, since python already has it implemented in the simple ```not```, ```and```, ```or```, and ```==``` operators (implication can be derived from these as well).
While its true that any propositional logic statement can be evaluated using Python, these statements cannot be stored in their unevaluated form (which is desirable to form a knowledge base).
Additionally, we cannot create variables without assigning them value in standard Python, but when we later want to iterate over all possible models the variables will need to take on many possible values.
Take the sentence ```a and b <=> c``` for example, we could write it in Python as ```(a and b) == c```, but we cannot store it and then simply retrieve it again in its original form.
And what if we don't know the values of ```a```, ```b```, and ```c``` to begin with (then it would give an error) and then later learn that ```b``` is False.
Python's standard implementation cannot deal with this.


So we will need to implement our own version of propositional logic in Python.
For variables we will use strings to represent them and later we can use dictionaries to represent models and then look up the value of each variable in that model.
For True and False we will simply use the existing ```True``` and ```False``` values in Python.
For all the operators we will implement classes that contain the logic sentences they operate on.


To begin with we have given you a skeleton for a class for the "and" operator which you need to complete and then make additional classes for all the other operators. For each class you need to implement ```__init__()``` to store the sub-sentences, ```__eq__()``` to check if two sentences are duplicates, and ```__repr__()``` to make a nice string representation of the sentence.


An additional option you might want to add is how to deal with repeated use of the same operation over multiple sentences.
If you wish you can implement a function that given a binary operator and a list of sentences chains them togehter (e.g. ```fun(And, [s1, s2, s3]) -> And(S1, And(s2, s3))```).
Another option is to allow your binary operators to take any number of inputs directly (e.g. ```And(s1, s2, s3)```).
In the latter case you will need to think about how to deal with the ```evaluation``` function in the next code block.

### Define Operators

#### Define Operator Class

In [38]:
from functools import cmp_to_key

class Operator:
    '''Define a general Operator class that all operators inherit'''

    def __init__(self, *args, symbol, is_unary):
        '''
            - Initialize all the arguments (can be more than 2)
            - Set the symbol that will be shown in __repr__
            - Specify if the operator is unary
            - Raise exception if the operator is unary and we have a different-than-one number of arguments
            - Raise exception if it is a n-ary operator and we have less than 2 arguments
        '''

        self.args = args
        self.symbol = symbol
        self.is_unary = is_unary

        if is_unary and len(args) != 1:
            raise Exception("This operator only has an argument")

        if not is_unary and len(args) < 2:
            raise Exception("This operator can only accept more than or equal to two arguments")

    # Check if two sentences are equal
    def __eq__(self, other):
        n = len(self.args)

        if not isinstance(other, self.__class__):
            return False

        if n != len(other.args):
            return False

        sort_fn = lambda x, y: 1 if str(x) < str(y) else -1

        sorted_self_args = sorted(self.args, key=cmp_to_key(sort_fn))
        sorted_other_args = sorted(other.args, key=cmp_to_key(sort_fn))

        for idx in range(n):
            if sorted_self_args[idx] != sorted_other_args[idx]:
                return False

        return True

    # Print the sentence
    def __repr__(self):
        first_arg = self.args[0]

        if self.is_unary:
            if isinstance(first_arg, str) or isinstance(first_arg, bool):
                return f"{self.symbol}{first_arg}"
            else:
                return f"{self.symbol}({first_arg})"

        no_args = len(self.args)
        return_string = None

        if isinstance(first_arg, str) or first_arg.is_unary:
            return_string = f"{first_arg}"
        else:
            return_string = f"({first_arg})"

        for idx in range(1, no_args):
            current_arg = self.args[idx]
            operand = None

            if isinstance(current_arg, str) or isinstance(current_arg, bool) or current_arg.is_unary:
                operand = current_arg
            else:
                operand = f"({current_arg})"

            return_string += f" {self.symbol} {operand}"

        return return_string

    # Hash function
    def __hash__(self):
        return hash(repr(self))

    def evaluate_sentence(self):
        raise NotImplementedError('The evaluate function has to be implemented in all specific operators')

#### Define evaluate argument function

In [39]:
# This evaluates each argument to either None or bool
# It can return a value directly or it can call the specific operator evaluation function

def evaluate_arg(arg, variables):
    if arg is None:
        return None

    elif isinstance(arg, bool):
        return arg

    elif isinstance(arg, str):
        if arg in variables:
            return variables[arg]
        else:
            return None

    return arg.evaluate_sentence(variables)

#### Define each specific operator

In [40]:
# Initialize all operators and define their evaluation function
# Identity operand = does not change the result when doing the operation

class And(Operator):
    def __init__(self, *args):
        super().__init__(*args, is_unary=False, symbol="⋀")

    def evaluate_sentence(self, variables):
        '''
            - We have to evaluate each arg
            - If the evaluated arg is False, return False
            - Else, use conjunction on the current result and the evaluated arg.
            - I initialized the result with the identity operand (True in this case)
        '''

        result = True

        for arg in self.args:
            evaluated_arg = evaluate_arg(arg, variables)

            if evaluated_arg is False:
                return False

            result = result and evaluated_arg

        return result

class Or(Operator):
    def __init__(self, *args):
        super().__init__(*args, is_unary=False, symbol="⋁")

    def evaluate_sentence(self, variables):
        '''
            - We have to evaluate each arg
            - If the evaluated arg is True, return True
            - Else, use disjunction on the current result and the evaluated arg.
            - I initialized the result with the identity operand (False in this case)
        '''

        result = False

        for arg in self.args:
            evaluated_arg = evaluate_arg(arg, variables)

            if evaluated_arg is True:
                return True

            result = result or evaluated_arg

        return result

class Equals(Operator):
    def __init__(self, *args):
        super().__init__(*args, is_unary=False, symbol="⇔")

    def evaluate_sentence(self, variables):
        '''
            - Evaluate first argument
                - If it's None, return None
            - Evaluate all other args one by one
                - If evaluated_arg is None, return None
                - If it's bool, check if the result is equal to the newly evaluated arg
        '''

        result = evaluate_arg(self.args[0], variables)

        if result is None:
            return None

        no_args = len(self.args)

        for idx in range(1, no_args):
            evaluated_arg = evaluate_arg(self.args[idx], variables)

            if evaluated_arg is None:
                return None

            result = (result == evaluated_arg)

        return result

class Implies(Operator):
    def __init__(self, *args):
        super().__init__(*args, is_unary=False, symbol="⇒")

    # __eq__ needs to be overridden by `Implies` because operation is not commutative
    def __eq__(self, other):
        n = len(self.args)

        if not isinstance(other, self.__class__):
            return False

        if n != len(other.args):
            return False

        for idx in range(n):
            if self.args[idx] != other.args[idx]:
                return False

        return True

    # __hash__ needs to be overridden everytime __eq__ is overridden
    def __hash__(self):
        return super().__hash__()

    def evaluate_sentence(self, variables):
        '''
            - Evaluate first argument
                - If it's None, return None
            - Evaluate all other args one by one
                - If the current result is False, we update the result to True (False can imply anything)
                - If evaluated_arg is None, return None
                - If it's bool, check if evaluated_arg is True
                (if we get here, then the current result is True
                 and only True implies True)
        '''

        result = evaluate_arg(self.args[0], variables)

        if result is None:
            return None

        no_args = len(self.args)

        for idx in range(1, no_args):
            if result is False:
                result = True
            else:
                evaluated_arg = evaluate_arg(self.args[idx], variables)

                if evaluated_arg is None:
                    return None

                result = (evaluated_arg == True)

        return result

class Not(Operator):
    def __init__(self, *args):
        super().__init__(*args, is_unary=True, symbol="¬")

    def evaluate_sentence(self, variables):
        '''
            - Evaluate the only argument
            - If it's none, return None
            - Else return not(evaluated_arg)
        '''

        evaluated_arg = evaluate_arg(self.args[0], variables)

        if evaluated_arg is None:
            return None

        return not evaluated_arg

#### Test sentences representation

In [41]:
print(And("a", "b", "c"))

print(Or("a", "b", "c"))

print(Equals("a", "b", "c"))

print(Implies("a", "b", "c"))

print(Not("a"))

print(Or("a", And("b", "c")))

print(Not(Or("a", "b")))

print(Equals("a", Not(Or("b", "c"))))

a ⋀ b ⋀ c
a ⋁ b ⋁ c
a ⇔ b ⇔ c
a ⇒ b ⇒ c
¬a
a ⋁ (b ⋀ c)
¬(a ⋁ b)
a ⇔ ¬(b ⋁ c)


#### Test \_\_eq\_\_

In [42]:
from unittest import main, TestCase

class TestEquality(TestCase):
    # applicable for disjunction as well
    def test_conjunction(self):
        self.assertTrue(And("a", "b") == And("b", "a"))
        self.assertFalse(And("a", "b") == And("b", Not("a")))

    def test_equality(self):
        self.assertTrue(Equals("a", "b", "c") == Equals("c", "a", "b"))
        self.assertFalse(Equals("a", Not("b"), "c") == Equals("c", "b", "a"))
        self.assertTrue(Equals(Or("a", "b"), "d", "w") == Equals("w", Or("a", "b"), "d"))

    def test_implication(self):
        self.assertTrue(Implies("a", "b", "c") == Implies("a", "b", "c"))
        self.assertFalse(Implies("a", "b", "c") == Implies("b", "a", "c"))
        self.assertFalse(Implies("a", "b", "c") == Implies(Not("a"), "b", "c"))

    def test_negation(self):
        self.assertTrue(Not("a") == Not("a"))
        self.assertFalse(Not("a") == Not("b"))
        self.assertTrue(Not(Or("a", "b")), Not(Or("b", "a")))

if __name__ == "__main__":
    main(argv=['first-arg-is-ignored'], exit=False)

.................
----------------------------------------------------------------------
Ran 17 tests in 0.028s

OK


### Evaluating propositional sentences in a given model

Being able to form sentences in propositional logic is not very useful if we cannot actually evaluate them.
For this we will implement a fuction ```evaluate``` that takes a sentence in propositional logic and a dictonary with the truth values of the different variables in a given model and returns the truth value of the sentence in that model.

In addition to the standard propositional logic we will add a new thing to the mix to make evaluation of large models more efficient down the line.
The thing we will add is the concept of an unknown or uninteresting truth value using the Python ```None```-type.
The idea with unknown values is that instead of iterating over every value for every variable to evaluate a model, we can decide to only iterate over some variables and set the others as unknown.
However if an important variable is set to unknown we might not be able to get the correct answer.
In order to protect against this the ```evaluate``` function will return ```True``` if the sentence is true, ```False``` if its false, and ```None``` if we cannot know since some variables are unknown to us.
In order to deal with the ```None``` values you will have to think what the answer to many different operators should be. For example, we know that ```And(False, None)``` should be ```False``` since ```And``` always returns ```False``` when at least one sub-sentence is ```False```.
You will also need to experiment with Python to see if the Python operators give the correct behavior. Does ```False == None``` return ```None```.
If not, how to deal with it.

A start of the function can be found below, but it is missing most things.
Additionally, the function for replacing the variables with their correct truth-value will currently give an error if a variable is missing, but we want it to return ```None``` to represent an unknown value instead.

In [43]:
def evaluate(sentence, variables):
    '''
    Evaluates the value of a logic sentences using the values given to the variables in a dictionary.
    Args:
        sentence (any): A boolean, None, a variable string, or an operator class containing subsentences
        variables (dict): A dictionary mapping variables represented by strings to boolean values
    Returns (bool/None): The truth value of the sentence with the given variables or None if the answer is unknown
    '''

    return evaluate_arg(sentence, variables)

#### Testing the evaluate function

We have now created a propositional logic grammar and a function to evaluate the truth value of sentences for a given model.
In order to make sure everything is correct we introduce a testing space below and ask you to implement and evaluate a number of sentences for a given model in order to see that you get the correct result.

Below you are given a model and are then supposed to implement and run evaluation on the sentences below, with the given expected results:

- $rain \leftrightarrow \neg sunny $ is ```True```
- $rain \rightarrow umbrella$ is ```True```
- $(\neg sunny \rightarrow \neg umbrella) \leftrightarrow False$ is ```True```
- $sunny \vee rain \rightarrow play$ is ```None```
- $sunny \rightarrow play$ is ```True```
- $\neg rain \leftrightarrow play$ is ```None```
- $\neg play$ is ```None```
- $sunny \wedge rain \wedge snow \leftrightarrow False$ is ```True```

In [44]:
test_model = {
    'rain':     True,
    'umbrella': True,
    'cloudy':   True,
    'sunny':    False,
}

class TestLogic(TestCase):
    def test_rain_equals_not_sunny_is_true(self):
        self.assertTrue(evaluate(Equals("rain", Not("sunny")), test_model))

    def test_rain_implies_umbrella_is_true(self):
        self.assertTrue(evaluate(Implies("rain", "umbrella"), test_model))

    def test_not_sunny_implies_not_umbrella_equals_false_is_true(self):
        self.assertTrue(evaluate(Equals(Implies(Not("sunny"), Not("umbrella")), False), test_model))

    def test_sunny_or_rain_implies_play_is_none(self):
        self.assertIsNone(evaluate(Implies(Or("sunny", "rain"), "play"), test_model))

    def test_sunny_implies_play_is_true(self):
        self.assertTrue(evaluate(Implies("sunny", "play"), test_model))

    def test_not_rain_equals_play_is_none(self):
        self.assertIsNone(evaluate(Equals(Not("rain"), "play"), test_model))

    def test_not_play_is_none(self):
        self.assertIsNone(evaluate(Not("play"), test_model))

    def test_sunny_and_rain_and_snow_equals_false_is_true(self):
        self.assertTrue(evaluate(Equals(And("sunny", "rain", "snow"), False), test_model))

if __name__ == "__main__":
    main(argv=['first-arg-is-ignored'], exit=False)

.................
----------------------------------------------------------------------
Ran 17 tests in 0.024s

OK


### Querying for possible models from a Knowledge Base

Finally we'll get to creating and dealing with knowledge bases.
Making a knowledge base is not that difficult, it's simply a collection of logical sentences that are known (or presumed) to be true in a given environment.

To create this collection we will use the Python ```set``` data structure.
The reason is that sets can only contain one of each element so we don't risk adding the same sentence multiple times.
Sets can be created by listing  a number of elements in curly brackets (```{}```), and elements can be added and removed with the ```.add()``` and ```.remove()``` functions.

The more difficult part is then creating a function for querying the knowledge base for whether a new sentence is true or not.
One simple way to do this query is to test every possible model (every possible combination of truth values for the variables).
For each model we figure out whether the query sentence is True or False, we then also check that all sentences in the knowledge base are True for that model.
Since we "know" that each sentence in the knowledge base should be true, any model that doesn't align with this can be discarded as an "invalid" model of our environment.
For all "valid" models we then return whether that model is True or False.

Of course many environments have a huge amount of variables which would then require us to check a huge number of models.
If we have $n$ variables in the environment we would need to check each sentence $n^2$ times.
However,  not every variable is necessarily relevant for our query sentence.
In the Wumpus World, for example, if there is a WUMPUS at [2,3] has no impact on whether there is a PIT at [1,1].
Using our knowledge of the environment we might be able to deduce the minimum number of variables that are relevant for our query and only test all combinations of them.
This is why we implemented the concept of unknown (```None```) values earlier.
Due to this we also consider a model "valid" if sentence in the knowledge base evaluate as unknown.
This comes with a risk of course, if we mistakenly exclude a relevant variable in our query a model that should be "invalid" might be considered "valid" since a False evaluation might now be unknown.
Additionally, we need to return all "valid" models that are unknown.

We can then use this query function to figure things about the environment.
If a query sentence only gives "valid" models in which it is True, then we know the sentence is True in the environment.
Conversely, if all returned models are False, the sentence is False in the environment.
If it returns some True models and some False models we know that, given what we know of the environment we cannot yet tell whether the sentence is True or not (in some advanced scenarios we might be able to use the number of True/False models to estimate the likelihood of the query being True/False in the actual environment).
If some of the returned models are unknown models, we know that there are relevant variables that were not considered in the query.
This likely means we have an error in the code that determines which variables should be relevant for a query.
Note that many variables that are not in the query sentence could still be relevant.
For example, if we query whether there is a PIT at [0,1] it is relevant if there is a breeze at [0,0].
Finally, if the function returns no valid models at all, this means that there are contradictions in the knowledge base and something has gone wrong in its construction.

Below you will find the skeleton of the query function for you to complete.

#### Define helper functions

In [45]:
from itertools import product

def generate_all_models(variables):
    models = []
    variables_tuple = tuple(variables)
    truth_value_combinations = product(*[(False, True)] * len(variables))

    for combination in truth_value_combinations:
        current_element = {}

        for idx, truth_value in enumerate(combination):
            current_variable = variables_tuple[idx]

            current_element[current_variable] = truth_value

        models.append(current_element)

    return models

def test_if_model_valid_in_KB(model, KB):
    for sentence in KB:
        if evaluate(sentence, model) is False:
            return False

    return True

#### Define query function

In [46]:
def query(query, KB, variables):
    '''
    For a given query and knowledge base, returns the models where all KB sentences are true
    in three lists depending on if the query is True, False, or None
    Args:
        query (any): Logic sentence to check the truth value of
        KB ({any}): A set of logic sentences, none of which should be False for the returned models
        variables ([str]): A list of strings that make up the relevant variables for the query
    Returns ([{str:bool}],[{str:bool}],[{str:bool}]): Lists of the "valid" models where the query was True, False, or None
    '''

    models = generate_all_models(variables)
    valid_models = [model for model in models if test_if_model_valid_in_KB(model, KB)]

    true_models = []
    false_models = []
    none_models = []

    for model in valid_models:
        evaluated_sentence = evaluate(query, model)

        if evaluated_sentence is True:
            true_models.append(model)
        elif evaluated_sentence is False:
            false_models.append(model)
        else:
            none_models.append(false_models)

    return (true_models, false_models, none_models)

#### Testing the querying function

No complex function is complete without testing.
To do this we will use the sentences from testing our evaluation function as the knowledge base.
You will be given a list of "relevant" variables and then you will query the knowledge base with the following sentences with the given expected results.

- $True$ returns 2 True models (and 0 False/None models).
- $rain \leftrightarrow \neg sunny $ returns 2 True models.
- $play \wedge cloudy$ returns 1 False model, and 1 unknown model.
- $sunny \leftrightarrow play$ returns 2 unknown models.
- $(sunny \leftrightarrow \neg cloudy) \leftrightarrow False$ returns 1 True and 1 False model.


In [47]:
KB = {
    Equals("rain", Not("sunny")),
    Implies("rain", "umbrella"),
    Equals(Implies(Not("sunny"), Not("umbrella")), False),
    Implies(Or("sunny", "rain"), "play"),
    Implies("sunny", "play"),
    Equals(Not("rain"), "play"),
    Not("play"),
    Equals(And("sunny", "rain", "snow"), False)
}

test_variables = {'rain', 'umbrella', 'cloudy', 'sunny'}

def test_query(query_arg):
    return query(query_arg, KB, test_variables)

class TestQuery(TestCase):
    def test_true(self):
        true_models, false_models, none_models = test_query(True)

        self.assertEqual(len(true_models), 2)
        self.assertEqual(len(false_models), 0)
        self.assertEqual(len(none_models), 0)

    def test_rain_equals_not_sunny(self):
        true_models, false_models, none_models = test_query(Equals("rain", Not("sunny")))

        self.assertEqual(len(true_models), 2)
        self.assertEqual(len(false_models), 0)
        self.assertEqual(len(none_models), 0)

    def test_play_and_cloudy(self):
        true_models, false_models, none_models = test_query(And("play", "cloudy"))

        self.assertEqual(len(true_models), 0)
        self.assertEqual(len(false_models), 1)
        self.assertEqual(len(none_models), 1)

    def test_sunny_equals_play(self):
        true_models, false_models, none_models = test_query(Equals("sunny", "play"))

        self.assertEqual(len(true_models), 0)
        self.assertEqual(len(false_models), 0)
        self.assertEqual(len(none_models), 2)

    def test_sunny_equals_not_cloudy_equals_false(self):
        true_models, false_models, none_models = test_query(Equals(Equals("sunny", Not("cloudy")), False))

        self.assertEqual(len(true_models), 1)
        self.assertEqual(len(false_models), 1)
        self.assertEqual(len(none_models), 0)

if __name__ == "__main__":
    main(argv=['first-arg-is-ignored'], exit=False)

.................
----------------------------------------------------------------------
Ran 17 tests in 0.026s

OK


## Using logic to understand the Wumpus World

Now that we have a working Propositional Logic, Knowledge Base, and Query system in place, we can finally start using it to create an agent.
Knowledge-based agents can come in many forms, but the most typical behavior is to dump all the relevant knowledge of the world into a knowledge base and then query the knowledge base for what action to perform next.
The percepts from the environment is then used to add new knowledge into the knowledge base to help with the next query.

Doing all of this can require very extensive knowledge bases and complex agents, which is beyond the scope of this assigment.
Instead we will use a very simple procedure for our agent:
- If the agent has the gold we know there must be safe path back to the entrance, take actions to follow that path back and then use CLIMB to finish the game.
- If there is glitter on the current tile, use the GRAB action to pick it up.
- Otherwise, query every tile for whether its safe or not.
    - If there is a safe tile that has not yet been visited, calculate a path to it and then follow that path.
        - If there is no path, calculate a path to the next unvisited safe tile.
    - If all safe tiles have been visited or are unreachable, find a path back to the start tile and then use CLIMB to finish the game.

As you can see this agent is very simple, not even considering using the SHOOT action, and assumes that all tiles that it doesn't know for sure to be unsafe.
For now though, this is enough.
Before we can start creating an agent, we need a knowledge base for it to query.
There are many things we know about the Wumpus World, many of which we could turn into logic and put into the knowledge base.
However, the more things we put in the knowledge base the longer the query will take, so we want to stick with only the essentials.
For example, we know that there can only be one WUMPUS and that each tile (except the start) has a 20% chance of being a PIT.
But the 20% chance is difficult to encode in propositional logic and not really relevant to our query, since we only care about things we know for sure.

So what do we put in the knowledge base?
We want to look for safe squares, so wee need to put a sentence for each tile defining what it means for that tile to be safe (there is no WUMPUS and no PIT there).
We also need some way to figure out whether there is a WUMPUS on a tile, so we need to add that having a WUMPUS on a tile means that the surrounding tiles all have stench.
The same goes for PIT and breeze.
Of course we also know some things about the starting tile.

Then we need a way to add new knowledge into the knowledge base based on the percepts that are encountered.

So now we need to create two functions.
One that returns the knowledge base about all relevant things we know at the start of each game, and one that given the percepts, the agent's position when those percepts were recieved, and a knowledge base, adds new relevant knowledge to the knowledge base.
A skeleton for each function is provided below

*Hint*: Remember that each sentence needs to be created for each tile and that each tile needs its own versions of all symbols. Try to think of a systematic way to name the symbols based on their tile coordinates.

### Helpers

In [48]:
def current_tile(position, item):
    x, y = position

    return f"{item}[{x}, {y}]"

def generate_adjacent_item_tiles(position, item):
    adj_tiles = adjacent_tiles(position[0], position[1])

    return list(map(lambda x: current_tile(x, item), adj_tiles))

def check_query_is_true(query_arg, KB, variables):
    _, false_models, unknown_models = query(query_arg, KB, variables)

    if len(unknown_models) != 0:
        raise Exception("Unknown models")

    return len(false_models) == 0

def replace_in_KB(old_sentence, new_sentence, KB):
    if old_sentence in KB:
        KB.remove(old_sentence)
        KB.add(new_sentence)

def _add_breeze_or_stench_knowledge(agent_pos, KB, percept, danger):
    current_percept = current_tile(agent_pos, percept)
    replace_in_KB(Not(current_percept), current_percept, KB)

    danger_positions = generate_adjacent_item_tiles(agent_pos, danger)
    adj_danger = Or(*danger_positions)

    if not adj_danger in KB:
        KB.add(adj_danger)

    unsafe_tiles = generate_adjacent_item_tiles(agent_pos, "SAFE")
    adj_visited_tiles = generate_adjacent_item_tiles(agent_pos, "VISITED")

    no_adj_tiles = len(unsafe_tiles)

    for idx in range(no_adj_tiles):
        tile_already_unsafe = Not(unsafe_tiles[idx]) in KB
        tile_not_visited = check_query_is_true(Not(adj_visited_tiles[idx]), KB, { adj_visited_tiles[idx] })

        if not tile_already_unsafe and tile_not_visited:
            replace_in_KB(unsafe_tiles[idx], Not(unsafe_tiles[idx]), KB)

def add_stench_knowledge(agent_pos, KB):
    _add_breeze_or_stench_knowledge(agent_pos, KB, "STENCH", "WUMPUS")

def add_breeze_knowledge(agent_pos, KB):
    _add_breeze_or_stench_knowledge(agent_pos, KB, "BREEZE", "PIT")

### Initial KB and changing KB

In [49]:
def initialWumpusKB():
    '''
    Returns a knowledge base of sentences known to be true in all initial Wumups Worlds
    Returns ({any}): A set of logic sentences
    '''

    KB = {
        Not("HAS_GOLD"),

        Not("PIT[0, 0]"),
        Not("WUMPUS[0, 0]"),
    }

    for i in range(4):
        for j in range(4):
            position = (i, j)

            pit_positions = generate_adjacent_item_tiles(position, "PIT")
            wumpus_positions = generate_adjacent_item_tiles(position, "WUMPUS")
            breeze_positions = generate_adjacent_item_tiles(position, "BREEZE")
            stench_positions = generate_adjacent_item_tiles(position, "STENCH")

            KB.add(current_tile(position, "SAFE"))
            KB.add(Not(current_tile(position, "VISITED")))

            KB.add(Not(current_tile(position, "GOLD")))

            KB.add(Equals(current_tile(position, "BREEZE"), Or(*pit_positions)))
            KB.add(Equals(current_tile(position, "STENCH"), Or(*wumpus_positions)))

            KB.add(Equals(current_tile(position, "PIT"), And(*breeze_positions)))
            KB.add(Equals(current_tile(position, "WUMPUS"), And(*stench_positions)))

            KB.add(Equals(current_tile(position, "SAFE"), Not(Or(current_tile(position, "PIT"), current_tile(position, "WUMPUS")))))
            KB.add(Implies(current_tile(position, "VISITED"), current_tile(position, "SAFE")))

    return KB

def addWumpusKnowledge(percepts, agent_pos, KB):
    '''
    Updates the given knowledge base by adding (and potentially removing) sentences from the knowledge base,
    based on the Wumpus World agent's position and perceptions.
    Args:
        percepts ([bool]): A list of whether each percept is active after the last action
        agent_pos (int, int): The tile currently occupied by the agent
        KB ({any}): The knowledge base of the current Wumpus World to be updated
    '''

    stench, breeze, glitter, _, _ = percepts

    current_visited = current_tile(agent_pos, "VISITED")
    replace_in_KB(Not(current_visited), current_visited, KB)

    if stench:
        add_stench_knowledge(agent_pos, KB)

    if breeze:
        add_breeze_knowledge(agent_pos, KB)

    if glitter:
        replace_in_KB(Not(current_tile(agent_pos, "GOLD")), current_tile(agent_pos, "GOLD"), KB)

### Creating the agent

With knowledge base in hand it's time to create our simple naive agent.

Reading the description of the agent ahead, you probably noticed that it requires a search algorithm to find a path from one point to another.
Since search algorithms aren't covered in this course a simple, implementation of the shortest path algorithm **A\*** has been implemented in the next code block for you to use.
It takes the start tile, goal tile, and a list of all safe tiles and then returns the next tile to walk to.
Even though search-algorithms like **A\*** are not covered in this course, you can read more about them in Chapter 3 (Solving Problems by Searching) of the course book (Artificial Intelligence: A Modern Approach).

You also recieve a function to run the game given a class to return actions (an agent), and a skeleton for the agent you will implement based on the earlier description.

#### A*

In [50]:
# An implementation of a priority queue that is necessary for calculating A*
import heapq

def a_star_on_tiles(start, goal, safe_tiles):
    '''
    Takes a start tile, end tile, and a list of safe tiles and returns the next step along the shortest path to the goal.
    Assumes that the world is a grid where each tile is connected to the adjacent tiles (not diagonally)
    Args:
        start (int, int): The coordinates of the start tile
        goal (int, int): The tile to find a shortest path towards
        safe_tiles [(int, int)]: A list of the tiles that can be traversed (should include start and goal)
    Returns ((int, int)/None): The next tile in the shortest path if one exists
    '''
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    # Run A* to get a dictionary containing for each tile, the previous tile along the path
    while len(frontier) > 0:
        distance, (x, y) = heapq.heappop(frontier)

        if (x,y) == goal:
            break

        for neighbor in [n for n in adjacent_tiles(x,y) if n in safe_tiles]:
            new_cost = cost_so_far[(x,y)] + 1 #Can be adapted to handle turning time
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + abs(x-neighbor[0]) + abs(y-neighbor[1])
                heapq.heappush(frontier, (priority, neighbor))
                came_from[neighbor] = (x,y)

    # Walk backward along the path until the first tile beyond the start is found
    # If the start tile is unreachable, return None
    current = goal
    while True:
        if current not in came_from:
            return None
        if came_from[current] == start:
            break
        current = came_from[current]

    return current

#### Play Game

In [51]:
def play_game(agent, show=True):
    '''
    Initiates and runs a random Wumpus World by calling the Agent.act() function to get actions
    Args:
        agent (any): An object which has implemented an act function
        show (bool): Whether to print the states of the game or not
    '''
    w = WumpusWorld()
    while not w.game_over:
        if show:
            w.show()

        w.actuate(agent.act(w.perceive(), w.agent_pos, w.agent_dir))

    if show:
        w.show()

#### SimpleAgent

#### Helpers

In [52]:
def generate_all_safe_tiles(KB):
    safe_tiles = []

    for i in range(4):
        for j in range(4):
            pos = (i, j)

            variables = { current_tile(pos, "SAFE"), current_tile(pos, "PIT"), current_tile(pos, "WUMPUS") }

            if check_query_is_true(current_tile(pos, "SAFE"), KB, variables):
                safe_tiles.append(pos)

    return safe_tiles

def get_correct_direction(position, next_tile):
    x, y = position
    next_x, next_y = next_tile

    if x < next_x:
        return RIGHT
    elif x > next_x:
        return LEFT

    # Same row
    if y < next_y:
        return UP

    return DOWN

def get_next_direction(current_direction, correct_direction):
    if current_direction == correct_direction:
        return None

    new_direction_going_left = current_direction - 1 if current_direction > 0 else 3
    new_direction_going_right = current_direction + 1 if current_direction < 3 else 0

    resulting_left = abs(correct_direction - new_direction_going_left)
    resulting_right = abs(correct_direction - new_direction_going_right)

    if resulting_left < resulting_right:
        return TURN_LEFT

    return TURN_RIGHT

def follow_path(position, direction, next_tile):
    correct_direction = get_correct_direction(position, next_tile)
    next_direction = get_next_direction(direction, correct_direction)

    # If next direction is None, go forward
    return next_direction or FORWARD

def go_back_to_start(position, direction, safe_tiles):
    if position == (0, 0):
        return CLIMB

    next_tile = a_star_on_tiles(position, (0, 0), safe_tiles)

    return follow_path(position, direction, next_tile)

#### Implementation

In [53]:
class SimpleAgent():
    '''
    Creates a simple knowledge-based agent for acting in the Wumpus World
    '''

    def __init__(self):
        self.KB = initialWumpusKB()

    def act(self, percepts, position, direction):
        '''
        Returns an action based on the agents percepts and position in this time step
            Args:
                percepts ([bool]): A list of, for each percept, whether it is active in this time step
                position (int, int): The coordinates of the tile the agent is currently in
                direction (int):  An int representing the direction the agent is currently facing
            Returns (str): A string representing the action to be taken (see the first cell for definitions)
        '''

        addWumpusKnowledge(percepts, position, self.KB)

        is_glitter = check_query_is_true(current_tile(position, "GOLD"), self.KB, { current_tile(position, "GOLD") })
        has_gold = check_query_is_true("HAS_GOLD", self.KB, { "HAS_GOLD" })
        safe_tiles = generate_all_safe_tiles(self.KB)

        # Found glitter
        if is_glitter and not has_gold:
            replace_in_KB(Not("HAS_GOLD"), "HAS_GOLD", self.KB)

            return GRAB

        # Has gold
        if has_gold:
            return go_back_to_start(position, direction, safe_tiles)


        # Find a safe tile that is not visited
        for safe_tile in safe_tiles:
            current_visited = current_tile(safe_tile, "VISITED")

            if check_query_is_true(Not(current_visited), self.KB, { current_visited }):
                next_tile = a_star_on_tiles(position, safe_tile, safe_tiles)

                if next_tile:
                    return follow_path(position, direction, next_tile)

        # No safe, non-visited, and reachable tile
        return go_back_to_start(position, direction, safe_tiles)

### Execute

In [55]:
# Execute this cell to play one game over the Wumpus World with the SimpleAgent

play_game(SimpleAgent())

				
g	w			
				
a				
Performance: 0
				
g	w			
				
a				
Performance: -1
				
g	w			
a				
				
Performance: -2
				
ga	w			
				
				
Performance: -3
				
a	w			
				
				
Performance: -4
				
a	w			
				
				
Performance: -5
				
a	w			
				
				
Performance: -6
				
	w			
a				
				
Performance: -7
				
	w			
				
a				
Performance: -8
				
	w			
				
				
Performance: 991
